In [10]:
import re
import os
import sentencepiece as spm
import polars as pl

In [5]:
# Download primeira parte do dataset
df = pl.read_parquet('hf://datasets/TucanoBR/GigaVerbo/data/train-00000-of-01573.parquet')

print(f"{df.shape[0]} linhas e {df.shape[1]} colunas ({", ".join(df.columns)})")

92372 linhas e 4 colunas (text, label, probs, metadata)


In [8]:
# Filtrar apenas textos com label 1 (texto de qualidade) e concatenar em uma única string
text_raw = "<|eos|>".join(
    df.filter(
        pl.col('label') == 1)
    .select(
        pl.col('text'))
    .to_series()
    .to_list()
)

with open('../data/gigaverbo.txt', 'w') as f:
    f.write(text_raw)

In [11]:
with open("../data/gigaverbo.txt", "r", encoding="utf-8") as f:
    text_raw = f.read()
    
print("Número de Caracteres:", len(text_raw))
print("Número de Palavras:", len(re.split(r"(\s)", text_raw)))
print("Número de Caracteres únicos:", len(set(text_raw)))
print("Número de Palavras únicas:", len(set(re.split(r"(\s)", text_raw))))

Número de Caracteres: 79444559
Número de Palavras: 25644299
Número de Caracteres únicos: 1265
Número de Palavras únicas: 748520


In [12]:
phrase = "Hoje é um belo dia"

# Criando vocabulario de caracteres
vocab_char = sorted(set(text_raw))

# Criando vocabulario de palavras
vocab_word = sorted(set(re.split(r"(\s)", text_raw)))

print("Frase:", phrase)
print("Frase em caracteres:", [vocab_char.index(c) for c in phrase])
print("Frase em palavras:", [vocab_word.index(w) for w in re.split(r"(\s)", phrase)])

Frase: Hoje é um belo dia
Frase em caracteres: [43, 82, 77, 72, 3, 155, 3, 88, 80, 3, 69, 72, 79, 82, 3, 71, 76, 68]
Frase em palavras: [179435, 3, 746180, 3, 724737, 3, 355836, 3, 432683]


In [18]:
VOCAB_SIZE = 4096
SPECIAL_TOKENS = ["<|eos|>"]

spm.SentencePieceTrainer.train(
    input="../data/gigaverbo.txt",              # Caminho para o arquivo de contendo o conteúdo para treinar o tokenizador
    model_prefix="../models/gigaverbo_tk",     # Prefixo do nome do arquivo de saída
    vocab_size=VOCAB_SIZE,                          # Tamanho do vocabulário
    model_type="bpe",                               # Tipo do tokenizador
    self_test_sample_size=0,                        # Tamanho do conjunto de teste (não utilizaremos)
    character_coverage=0.995,                       # Fração de caracteres que devem ser cobertos pelo modelo
    input_format="text",                            # Formato do arquivo de entrada
    num_threads=os.cpu_count(),                     # Número de threads a serem utilizadas
    split_digits=True,                              # Separar os dígitos tratando-os como tokens separados
    allow_whitespace_only_pieces=True,              # Permitir tokens que representem apenas espaços em branco
    byte_fallback=True,                             # Utilizar tokenização de bytes como fallback
    unk_surface="<|unk|>",                          # Representação do token desconhecido
    normalization_rule_name="identity",             # Regra de normalização (identity = sem normalização)
    user_defined_symbols=SPECIAL_TOKENS             # Adiciona tokens especiais definidos pelo usuário
)

sentencepiece_trainer.cc(78) LOG(INFO) Starts training with : 
trainer_spec {
  input: ../data/gigaverbo.txt
  input_format: text
  model_prefix: ../models/gigaverbo_tk
  model_type: BPE
  vocab_size: 4096
  self_test_sample_size: 0
  character_coverage: 0.995
  input_sentence_size: 0
  shuffle_input_sentence: 1
  seed_sentencepiece_size: 1000000
  shrinking_factor: 0.75
  max_sentence_length: 4192
  num_threads: 8
  num_sub_iterations: 2
  max_sentencepiece_length: 16
  split_by_unicode_script: 1
  split_by_number: 1
  split_by_whitespace: 1
  split_digits: 1
  pretokenization_delimiter: 
  treat_whitespace_as_suffix: 0
  allow_whitespace_only_pieces: 1
  user_defined_symbols: <|eos|>
  required_chars: 
  byte_fallback: 1
  vocabulary_output_piece_score: 1
  train_extremely_large_corpus: 0
  seed_sentencepieces_file: 
  hard_vocab_limit: 1
  use_all_vocab: 0
  unk_id: 0
  bos_id: 1
  eos_id: 2
  pad_id: -1
  unk_piece: <unk>
  bos_piece: <s>
  eos_piece: </s>
  pad_piece: <pad>
  unk_

er_interface.cc(425) LOG(INFO) Adding meta_piece: <0x7F>
trainer_interface.cc(425) LOG(INFO) Adding meta_piece: <0x80>
trainer_interface.cc(425) LOG(INFO) Adding meta_piece: <0x81>
trainer_interface.cc(425) LOG(INFO) Adding meta_piece: <0x82>
trainer_interface.cc(425) LOG(INFO) Adding meta_piece: <0x83>
trainer_interface.cc(425) LOG(INFO) Adding meta_piece: <0x84>
trainer_interface.cc(425) LOG(INFO) Adding meta_piece: <0x85>
trainer_interface.cc(425) LOG(INFO) Adding meta_piece: <0x86>
trainer_interface.cc(425) LOG(INFO) Adding meta_piece: <0x87>
trainer_interface.cc(425) LOG(INFO) Adding meta_piece: <0x88>
trainer_interface.cc(425) LOG(INFO) Adding meta_piece: <0x89>
trainer_interface.cc(425) LOG(INFO) Adding meta_piece: <0x8A>
trainer_interface.cc(425) LOG(INFO) Adding meta_piece: <0x8B>
trainer_interface.cc(425) LOG(INFO) Adding meta_piece: <0x8C>
trainer_interface.cc(425) LOG(INFO) Adding meta_piece: <0x8D>
trainer_interface.cc(425) LOG(INFO) Adding meta_piece: <0x8E>
trainer_inter

In [13]:
phrase = "Olá Mundo, tudo bem?<|eos|>"

tk = spm.SentencePieceProcessor(model_file="../models/gigaverbo_tk.model") # Instanciando o tokenizador treinado

phrase_encode = tk.encode(phrase)
phrase_tokens = tk.encode_as_pieces(phrase)
phrase_decode = tk.decode(phrase_encode)

print("Frase Tokenizada:", phrase_tokens)
print("Frase tokenizada:", phrase_encode)
print("Frase reconstruída:", phrase_decode)

Frase Tokenizada: ['▁Ol', 'á', '▁Mu', 'ndo', ',', '▁tudo', '▁bem', '?', '<|eos|>']
Frase tokenizada: [3188, 4047, 2362, 341, 4040, 1927, 824, 4091, 3]
Frase reconstruída: Olá Mundo, tudo bem?<|eos|>


In [22]:
text_tokenized = tk.encode(text_raw)

# Salvando o texto tokenizado
with open("../data/gigaverbo_tokenized.txt", "w", encoding="utf-8") as f:
    f.write(" ".join([str(t) for t in text_tokenized]))

print("Amostra do texto tokenizado:", text_tokenized[:20])

Amostra do texto tokenizado: [2762, 1993, 596, 900, 271, 613, 445, 2710, 845, 4002, 372, 3651, 3479, 4091, 14, 14, 2385, 2258, 2188, 269, 1041, 1332, 4040, 2001, 1110, 374, 310, 1456, 739, 3763, 261, 866, 513, 301, 486, 547, 4037, 951, 1556, 296, 1294, 1064, 301, 888, 261, 1045, 1087, 4040, 323, 435, 298, 4036, 614, 302, 607, 1354, 596, 900, 271, 613, 733, 4030, 914, 445, 2710, 845, 4002, 372, 3651, 3479, 4040, 595, 323, 935, 1071, 1146, 2443, 362, 262, 1401, 297, 451, 269, 765, 4037, 14, 14, 4055, 4031, 885, 4040, 367, 2710, 845, 4002, 1339, 453, 1803, 828, 769]
